In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import os
import pix2pix
import cv2
import matplotlib.pyplot as plt
from pix2pix import models
import tqdm
from pix2pix import config

In [ ]:
gt_path = 'videos/input_video_gt.mp4'
noisy_path = 'videos/input_video_noisy.mp4'
pred_save_path = 'videos/predicted_video_v2.mp4'

# device = "cuda" if torch.cuda.is_available() else "cpu"
device = 'cpu'

In [ ]:
state_dict = torch.load('../../models/pix2pix_32_11112021_051302/checkpoints/gen_epoch_4_pix2pix_32_11112021_051302.pt')
model = models.unet(norm_layer=torch.nn.InstanceNorm2d, **config.model.GEN_ARGS)
model = torch.nn.DataParallel(model)
model.load_state_dict(state_dict)
model.eval();

In [ ]:
class img_dataset(torch.utils.data.Dataset):
    def __init__(self, imgs):
        self.imgs = imgs

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.to_list()
        
        inp = torch.Tensor(self.imgs[idx])
        inp = (inp / (255 / 2)) - 1
        inp = inp.permute((2, 0, 1))
        
        return inp

In [6]:
%%time

noisy_cap = cv2.VideoCapture(noisy_path)
fps = noisy_cap.get(cv2.CAP_PROP_FPS)
width = int(noisy_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(noisy_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Be sure to use lower case
out = cv2.VideoWriter(pred_save_path, fourcc, fps, (width, height))
    
frame_list = []

while noisy_cap.isOpened():
    ret, frame = noisy_cap.read()
    
    if ret == False:
        break
        
    frame = cv2.resize(frame, (256, 256))
    frame_list.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
noisy_cap.release()
img_dataloader = torch.utils.data.DataLoader(img_dataset(frame_list), 
                                             batch_size = 16,
                                             shuffle = False)


for ip in tqdm.tqdm(img_dataloader, total = len(img_dataloader)):
    ip = ip.to(device)
    preds = model(ip).detach()
    preds = (preds + 1) / 2 
    preds = preds.permute(0, 2, 3, 1)
    preds = preds.to('cpu')
    preds = (preds.numpy() * 255).astype('uint8')
    
    for i in range(preds.shape[0]):
        frame_to_save = cv2.resize(preds[i, :, :, :], (width, height))
        out.write(cv2.cvtColor(frame_to_save, cv2.COLOR_RGB2BGR))
        
out.release()

100%|██████████| 94/94 [00:14<00:00,  6.43it/s]

CPU times: user 43.7 s, sys: 5.7 s, total: 49.4 s
Wall time: 15.4 s
